# GRADED ASSIGNMENT: Implementing sudoku puzzle solver

**The notebook is part of a graded coursework assignment for your unit. Full instructions and specifications of the coursework as a whole are in the accompanying pdf.** 

## Introduction

A sudoku puzzle is a 9x9 grid with some fixed values. To solve the puzzle, you need to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

In this notebook, you write your implementation of an agent to solve sudoku puzzles for submission as part of your coursework i.e., solver implementation. 

Here is a reminder of the key rules associated with the implementation:

 * You can use any algorithm you like, from the course material or otherwise, provided you can implement it yourself from scratch.
   
 * The assessment criteria of your solver implementation will be **correctness** and **speed** of sudoku puzzle solving.

   
 * The implementation will be assessed using **automated testing**. It is crucial for you to ensure that your implementation is compatible with the automarker by following the input/output specifications of the code template given in the notebook. The compatibility of your solution to the automarker must be tested using pre-submission tests (detailed in this notebook under **Phase Two: Essential Pre-submission Tests**). Failure to ensure compatibility to the automarker using the procedure in this notebook will result in a zero for the implementation.

   
 * You can test correctness of your implementation using the set of puzzles of various difficulty provided with this assignment. Automated testing will also include additional (hidden) puzzles which you are not given access to.


## Preliminaries: loading the puzzle data

To get started, the cell below will load in some sample sudoku puzzles for you to see the format. There are sudokus of various difficulty levels provided: easier sudokus typically have more pre-filled cells. The cell below illustrates how to load the puzzles using the easiest difficulty level as an example. However, you can load a puzzle of any difficulty from the provided set by changing the input file. The test cell in **Phase Two: Essential Pre-submission Tests** automates the process.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty level in the provided set comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [16]:
import numpy as np

# Load sudokus
sudoku = np.load("data/easy_puzzle.npy")
print("easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[8 5 2 9 7 6 2 4 3]
 [6 7 9 1 4 3 2 8 5]
 [0 3 1 2 5 8 7 6 9]
 [3 1 4 5 2 7 8 9 6]
 [7 6 8 3 9 1 4 5 0]
 [9 2 5 6 0 0 3 7 1]
 [5 4 3 8 6 2 9 1 7]
 [1 9 7 4 3 5 0 2 8]
 [2 8 6 7 1 9 5 3 4]] 

Solution of first sudoku:
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]


## Phase One: Implementation
Your solver implementation must follow the template in the code cell below. Specifically, there must be
a function called `sudoku_solver(sudoku)`, which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested by the automarker.

All sudoku puzzles in this assignment will have either a single solution or no solution. If a puzzle has no solution, your `sudoku_solver(sudoku)` function must return a 9x9 NumPy array whose values are all equal to -1.

Start writing your implementation in the code cell below following the template. If necessary, you may add new cells, provided you do not duplicate or copy-paste cells. The notebook must be self-contained without needing to import any external `.py` files.

You must follow the following rules to avoid breaking the automarker script: <br/>

**Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.** <br/>

**Do not duplicate or copy/paste cells in the notebook.**  Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary. <br/>

**Remember to save your work regularly, and double-check you are submitting the correct version.**<br/>


In [1]:
import numpy as np

class Sudoku:
    
    def __init__(self, grid):
        self.grid = grid
    
    def domains_search(self, r, c):
    
        domains = list(range(1, 10))

        for value in self.grid[r]:
            if value in domains:
                domains.remove(value)
        
        for i in range(9):
            if self.grid[i][c] in domains:
                domains.remove(self.grid[i][c])
                
        for i in range(r // 3 * 3, r // 3 * 3 + 3):
            for j in range(c // 3 * 3, c // 3 * 3 + 3):
                if self.grid[i][j] in domains:
                    domains.remove(self.grid[i][j])
        
        return domains
        
    def MRV(self):
        
        least = 10
        variable = None
        
        for r in range(9):
            for c in range(9):
                if self.grid[r][c] == 0:
                    domains = self.domains_search(r, c)
                    num = len(domains)
                    
                    if num == 0:
                        return (r, c, domains)

                    if num < least:
                        least = num
                        variable = (r, c, domains)
                        
                        if num == 1:
                            return variable
        
        return variable
    
    def validated(self):
        
        for r in range(9):
            for c in range(9):
                if self.grid[r][c] != 0:
                    temp = self.grid[r][c]
                    self.grid[r][c] = 0 
                    valid = self.domains_search(r, c)
                    
                    if temp not in valid:
                        return False    
                    
                    self.grid[r][c] = temp
                    
        return True
        
    def solver(self):

        optimum = self.MRV()
        
        if optimum is None:
            return True

        r, c, domains = optimum
        
        if len(domains) == 0:
            return False

        for domain in domains:
            self.grid[r][c] = domain
            
            if self.solver():
                return True
                
            self.grid[r][c] = 0
        
        return False

def sudoku_solver(problem):
    
    puzzle = Sudoku(problem)
    
    if not puzzle.validated():
        return np.full((9, 9), -1)
    
    if puzzle.solver():
        return puzzle.grid
        
    else:
        return np.full((9, 9), -1)

**ALL YOUR IMPLEMENTATION MUST BE ABOVE THIS POINT (IN PHASE ONE). PHASE TWO IS FOR VALIDATION ONLY.**

## Phase Two: Essential Pre-submission Tests

**Step 1: Validation on provided puzzles** <br/>
Use the cell below to validate the performance of your solver implementation on the entire set of provided sudoku puzzles of various difficulty levels. There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

To enable the validation cell, set the constant `SKIP_TESTS` to `False`. If your code fails any tests of one difficulty, the code will stop, but you can modify this behaviour if it is helpful for your validation process.

Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

In [2]:
SKIP_TESTS = True

if not SKIP_TESTS:
    import time
    import numpy as np
    __SCORES = {}
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            if not isinstance(your_solution, np.ndarray):
                print("\033[91m[ERROR] Your sudoku_solver function returned a variable that has the incorrect type. If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns a {} object when {} was expected.\n\x1b[m".format(type(your_solution), np.ndarray))
            elif not np.all(your_solution.shape == (9, 9)):
                print("\033[91m[ERROR] Your sudoku_solver function returned an array that has the incorrect shape.  If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns an array with shape {} when {} was expected.\n\x1b[m".format(your_solution.shape, (9, 9)))
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took {} seconds to solve.\n".format(end_time-start_time))

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        __SCORES[difficulty] = {
            'correct': count,
            'total': len(sudokus)
        }

**Step 2: Check compatibility with the automarker** <br/>
The following cell tests if your implementation is compatible with the automarker and ready for submission. 

**You must not skip this step!**

**IMPORTANT**: First, you must set `SKIP_TESTS` in the validation cell above back to `True`. Make sure it stays set to `True` in the notebook you submit.

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). 

Now look at the output of the cell below. <br/>
The following message indicates that no incompatibility issues with the automarker have been detected and you can submit your work:
 
 <center>Congratulations!  <br/>                                                                
Your work meets all the required criteria <br/>        
    and is ready for submission.  <br/> </center>

<br/>

**If you do not see the message above in the output, then your submission is not ready.** This could be your code is still running (did you forget to set `SKIP_TESTS` to `True`?) or there has been an error. You need to go back and debug to ensure compatibility with the automarker, or else a zero will be awarded for the implementation.


In [3]:
import sys
import pathlib

fail = False;

success = '\033[1;32m[✓]\033[0m'
issue = '\033[1;33m[!]'
error = '\033[1;31m\t✗'
indent_success = '\033[1;32m\t✓'

#######
##
## Skip Tests check.
##
## Test to ensure the SKIP_TESTS variable is set to True to prevent it slowing down the automarker.
##
#######

if not SKIP_TESTS:
    fail = True;
    print("{} \'SKIP_TESTS\' is incorrectly set to False.\033[0m".format(issue))
    print("{} You must set the SKIP_TESTS constant to True in the cell above.\033[0m".format(error))
else:
    print('{} \'SKIP_TESTS\' is set to true.\033[0m'.format(success))

#######
##
## Report File Check.
##
## Test that checks there is a report pdf file found in the same folder as the notebook. This is required by the coursework specification.
##
#######

p1 = pathlib.Path('./report.pdf')
p2 = pathlib.Path('./Report.pdf')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("{} Report PDF not found.\033[0m".format(issue))
    print("{} You must include a separate file called report.pdf in your submission.\033[0m".format(error))
else:
    print('{} Report PDF found.\033[0m'.format(success))

#######
##
## File Name check.
##
## Test to ensure file has the correct name. This is important for the marking system to correctly process the submission.
##
#######
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("{} The notebook name is incorrect.\033[0m".format(issue))
    print("{} This notebook file must be named sudoku.ipynb\033[0m".format(error))
else:
    print('{} The notebook name is correct.\033[0m'.format(success))

#######
##
## Create classifier function check.
##
## Test that checks the create_classifier function exists. The function should train the classifier and return it so that it can be evaluated by the marking system.
##
#######

if "sudoku_solver" not in dir():
    fail = True;
    print("{} The sudoku_solver function has not been defined.\033[0m".format(issue))
    print("{} Your code must include a sudoku_solver function as described in the coursework specification.\033[0m".format(error))
    print("{} If you believe you have, \'restart & run-all\' to clear this error.\033[0m".format(error))
else:
    print('{} The sudoku_solver function has been defined.\033[0m'.format(success))



try:
    _sudoku = np.load("data/very_easy_puzzle.npy")[0]
    _solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(_sudoku), _solution):
        print("{} Your sudoku_solver function does not correctly solve the first sudoku.\033[0m".format(issue))
        print()
        print("{} Your assignment is unlikely to get any marks from the autograder. While we will\033[0m".format(error))
        print("{} try to check it manually to assign some partial credit, we encourage you to ask\033[0m".format(error))
        print("{} for help on the forum or directly to a tutor.\033[0m".format(error))
        print()
        print("{} Please use the report file to explain your code anyway.\033[0m".format(error))
    else:
        print("{} Your sudoku_solver function correctly solves the first sudoku.\033[0m".format(success))
        if "__SCORES" in dir():
#             print("{} Test set summary - Not Found.\033[0m".format(issue))
#             print("{} Test set summary could not be found. This is automatically generated when the \033[0m".format(error))
#             print("{} above test cell is run. If you would like to see the summary please run the above cell.\033[0m".format(error))
#             print("{} You do not need this for submission, it is only for your convenience.\033[0m".format(error))
#         else:
            correct = 0
            total = 0
            for key, value in __SCORES.items():
                correct += value['correct']
                total += value['total']
                
            print("{} Test set summary - {}/{} Correct.\033[0m".format(issue, correct, total))
            if total != correct:
                
                for key, value in __SCORES.items():
                    if value['correct'] == value['total']:
                        print("{} {}/{} of {} sudokus correct.\033[0m".format(indent_success, value['correct'], value['total'], key))
                    else:
                        print("{} {}/{} of {} sudokus correct.\033[0m".format(error, value['correct'], value['total'], key))
            
except Exception as e:
    fail = True
    print("{} Error running test set.\033[0m".format(issue))
    print("{} Your code produced the following error. This error will result in a zero from the automarker, please fix.\033[0m".format(error))
    print(e)

    

#######
##
## Final Summary
##
## Prints the final results of the submission tests.
##
#######

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("\033[1m\n\n")
    print("╔═══════════════════════════════════════════════════════════════╗")
    print("║                        Congratulations!                       ║")
    print("║                                                               ║")
    print("║            Your work meets all the required criteria          ║")
    print("║                   and is ready for submission.                ║")
    print("╚═══════════════════════════════════════════════════════════════╝")
    print("\033[0m")
    

[✓] 'SKIP_TESTS' is set to true.
[✓] Report PDF found.
[✓] The notebook name is correct.
[✓] The sudoku_solver function has been defined.
[✓] Your sudoku_solver function correctly solves the first sudoku.



╔═══════════════════════════════════════════════════════════════╗
║                        Congratulations!                       ║
║                                                               ║
║            Your work meets all the required criteria          ║
║                   and is ready for submission.                ║
╚═══════════════════════════════════════════════════════════════╝



In [5]:
# This is a TEST CELL. Do not delete or change.